#Mounting Drive

In [ ]:
#@title

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#INPUT USER FOR YOUTUBE SCRAPING

In [ ]:
fileName = input("Input Name File.csv :  ")
youtubeUrl = input("Input your Youtube URL :   ")

Input Name File.csv :  toyota.csv
Input your Youtube URL :   https://www.youtube.com/watch?v=aWbvhUplLcE&t=8s


# Data Scraping


In [ ]:
import pandas as pd
import json
import os
import sys
import re
import time

import requests

# pandas dataframe display configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_service_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
# Input csv file name
FILE_NAME = str(fileName) ###NAMA FILE###

# set parameters
# filter comments by popularity or recent, 0:False, 1:True
SORT_BY_POPULAR = 0
# default recent
SORT_BY_RECENT = 1

#Input Total Comment
#x = input("Input Total Commnent =  ")
COMMENT_LIMIT = 99999999999 ##TOTAL COMMENT##

YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'


def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default


def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']
    
    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token']}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)


def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)

    html = response.text
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:
        return # Unable to extract configuration
    if language:
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language

    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))

    section = next(search_dict(data, 'itemSectionRenderer'), None)
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:
        # Comments disabled?
        return

    needs_sorting = sort_by != SORT_BY_POPULAR
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)

        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        if needs_sorting:
            sort_menu = next(search_dict(response, 'sortFilterSubMenuRenderer'), {}).get('subMenuItems', [])
            if sort_by < len(sort_menu):
                continuations = [sort_menu[sort_by]['serviceEndpoint']]
                needs_sorting = False
                continue
            raise RuntimeError('Failed to set sorting')

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    # Process continuations for comments and replies.
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # Process the 'Show more replies' button
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])

        for comment in reversed(list(search_dict(response, 'commentRenderer'))):
            yield {'cid': comment['commentId'],
                   'text': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]),
                   'time': comment['publishedTimeText']['runs'][0]['text'],
                   'author': comment.get('authorText', {}).get('simpleText', ''),
                   'channel': comment['authorEndpoint']['browseEndpoint'].get('browseId', ''),
                   'votes': comment.get('voteCount', {}).get('simpleText', '0'),
                   'photo': comment['authorThumbnail']['thumbnails'][-1]['url'],
                   'heart': next(search_dict(comment, 'isHearted'), False)}

        time.sleep(sleep)

#def choosePath():
  #from google.colab import files
  #drive.mount('/content/drive')
  #set_input= input("masukan directory(")




def search_dict(partial, search_key):
    stack = [partial]
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)


def main(url):
    df_comment = pd.DataFrame()
    try:
        youtube_url = url
        limit = COMMENT_LIMIT

        print('Downloading Youtube comments for video:', youtube_url)

        count = 0

        start_time = time.time()

        for comment in download_comments(youtube_url):

            df_comment = df_comment.append(comment, ignore_index=True)

            # comments overview
            comment_json = json.dumps(comment, ensure_ascii=False)
            print(comment_json)

            count += 1

            if limit and count >= limit:
                break

        print("DataFrame Shape: ",df_comment.shape,"\nComment DataFrame: ", df_comment)

        if not os.path.isfile(FILE_NAME):
            df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
        else:  # else it exists so append without writing the header
            df_comment.to_csv(FILE_NAME, mode='a', encoding='utf-8', index=False, header=False)

        print('\n[{:.2f} seconds] Done!'.format(time.time() - start_time))

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)                            


# dumping youtube comments

""" 
1. Dump comments to a csv  from a single video

"""

youtube_URL = str(youtubeUrl)
main(youtube_URL)

"""
2. Dump comments to a csv by parsing links from a csv with video links

Example -
Create a csv with one column titled 'link'
a sample is given below

'ytb_video_list.csv'

link



"""
# df_video_list = pd.read_csv('ytb_video_list.csv')
# print(df_video_list['link'].map(lambda x: main(x)))
# print(main(pd.read_csv('ytb_video_list.csv')['link']))


"""
3. Dump to a csv from a a list with video links
"""

#ytb_video_list = ['https://www.youtube.com/watch?v=NaeErfFmH4A',
                  #]

#for video_link in ytb_video_list:
    #main(video_link)

Streaming output truncated to the last 5000 lines.
{"cid": "UgwsBWWgnrJCw4xRlvB4AaABAg", "text": "Mantap mas review nya", "time": "6 months ago", "author": "Tri Widodo(PT.Jvc)", "channel": "UCAEFwxatXRPcJ3L5khPQ0Hg", "votes": "0", "photo": "https://yt3.ggpht.com/Zn5X7kRjLH-xlERm5kMheYjLR758J9XWklAGpLhtrquwubfc4z7Kn7RAo1J2dTiJBSopIq78wg=s176-c-k-c0x00ffffff-no-rj", "heart": false}
{"cid": "UgzteLbAg4WheXbvtT54AaABAg", "text": "😍👍👍👍🙏", "time": "6 months ago", "author": "Tri Widodo(PT.Jvc)", "channel": "UCAEFwxatXRPcJ3L5khPQ0Hg", "votes": "0", "photo": "https://yt3.ggpht.com/Zn5X7kRjLH-xlERm5kMheYjLR758J9XWklAGpLhtrquwubfc4z7Kn7RAo1J2dTiJBSopIq78wg=s176-c-k-c0x00ffffff-no-rj", "heart": false}
{"cid": "UgyOCrYhWXZHuu0FO6t4AaABAg", "text": "Mudah²an bisa kebeli. Aamiin🤲", "time": "6 months ago", "author": "Syahrial Effendy", "channel": "UCt25tTq3IqJz6chpp_kqjZA", "votes": "1", "photo": "https://yt3.ggpht.com/ytc/AKedOLR4UBKlTIHFEIj3BySgmj0OCKiNFNOONAPeTGmE=s176-c-k-c0x00ffffff-no-rj", "hear

'\n3. Dump to a csv from a a list with video links\n'

#Data Cleansing

In [ ]:
!pip install nltk
!pip install Sastrawi
!pip install googletrans

     |████████████████████████████████| 209 kB 5.2 MB/s 
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 1.3 MB 10.3 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15734 sha256=9d1036ff312f72b2d2ebb61af995080083155d6d772f82bd070e6ac9298dc372
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import pytz 
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import ast
import string
from wordcloud import WordCloud
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from googletrans import Translator
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Define the.csv file into dataframe
df = pd.read_csv(FILE_NAME)
df.head()

,author,channel,cid,heart,photo,text,time,votes
0,AutonetMagz,UCvt4e--lfkOCvC_RotjXh-g,UgwB-Nd0T3YO5itaQAR4AaABAg,0.0,https://yt3.ggpht.com/ytc/AKedOLTzkX1Eg5ECosO_...,Sah! KIA Sonet & Nissan Magnite sekarang punya...,7 months ago,1.3K
1,BocilKudet,UCwfmUoYHPgMyjmUDG122isw,Ugx3b92JcPzXAl0bA8t4AaABAg,0.0,https://yt3.ggpht.com/m65eURd1NQDC0HfisUSDUlbT...,"Mirip mazda 2 all new yaa, mantap👍🏻",14 hours ago,0
2,Imas Sartika,UC3EdLxD0c2z8k--ZRlUdY8A,UgzlfrqKSNqAWDZqv5F4AaABAg,0.0,https://yt3.ggpht.com/ytc/AKedOLRh5SgRtmWH6hRd...,semoga 4 tahun lagi 🤭🤭🤣 sholawatin aja dulu 🤲,17 hours ago,0
3,Ismail Abd Hakim,UC4G-bdp2qUGrCo5M5MZXp6g,Ugzc3yPwr-8TAbfdyE14AaABAg,0.0,https://yt3.ggpht.com/ytc/AKedOLTISKAuMqN3xqZ8...,kembar ativa di malaysia,3 days ago,0
4,Bakul Iwak,UCtCya5-CYbq1ZoS5htnoxQg,UgzP5lK52fSvwc1KTNJ4AaABAg,0.0,https://yt3.ggpht.com/ytc/AKedOLQvKc7q9wBkXRr5...,"Saya udah beli ini mobil, dan puas sama fitur,...",3 days ago,0


In [ ]:
#to define if the dataframe does have a null content, if it is run the code {NULL CONTENT}
df.info()
rows, columns = df.shape
cell_count = rows * columns
number_of_nulls = df.isnull().sum().sum()
percentage_of_missing = (number_of_nulls / cell_count) * 100
print(f'Percentage of missing values: {percentage_of_missing}%')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5586 entries, 0 to 5585
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   author   5586 non-null   object 
 1   channel  5586 non-null   object 
 2   cid      5586 non-null   object 
 3   heart    5586 non-null   float64
 4   photo    5586 non-null   object 
 5   text     5586 non-null   object 
 6   time     5586 non-null   object 
 7   votes    5586 non-null   object 
dtypes: float64(1), object(7)
memory usage: 349.2+ KB
Percentage of missing values: 0.0%


In [ ]:
#Dropping colloum the data doesn't needed and 
to_drop = ['channel',
            'cid',
            'heart',
            'photo',
            ]
df.drop(to_drop, inplace=True, axis=1)
df = df.rename(columns={'author': 'channel', 'text': 'comment', 'votes': 'like'})
df.head()

,channel,comment,time,like
0,AutonetMagz,Sah! KIA Sonet & Nissan Magnite sekarang punya...,7 months ago,1.3K
1,BocilKudet,"Mirip mazda 2 all new yaa, mantap👍🏻",14 hours ago,0
2,Imas Sartika,semoga 4 tahun lagi 🤭🤭🤣 sholawatin aja dulu 🤲,17 hours ago,0
3,Ismail Abd Hakim,kembar ativa di malaysia,3 days ago,0
4,Bakul Iwak,"Saya udah beli ini mobil, dan puas sama fitur,...",3 days ago,0


In [ ]:
#Using this to delete row in column comment that doesnt have alphabetical
df = df[df.comment.str.contains(r'[^a-zA-Z]')]

#Using this to delete row in column comment that only cointans  3 alphabetical
df.drop(df[(df['comment'].map(len) < 4 )].index, inplace=True)

#using this to delete same channel
df = df.drop_duplicates()

#using this to set every word on comment colum to lowercase
df['comment'] = df['comment'].str.lower()

df['pertanyaan'] = np.where(df.comment.str.contains(r'[?]'), df.comment, False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5208 entries, 0 to 5585
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel     5208 non-null   object
 1   comment     5208 non-null   object
 2   time        5208 non-null   object
 3   like        5208 non-null   object
 4   pertanyaan  5208 non-null   object
dtypes: object(5)
memory usage: 244.1+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
'''
##to combine word for "tidak"
for i in range(len(df)):
  if 'tidak' in df['comment']:
    split_text = df['comment'][i].split()
    for a,b in enumerate(split_text):
      if b == 'tidak':
        split_text[a:(a+2)] = [''.join(split_text[a:(a+2)])]
    df['comment'][i] = ' '.join(split_text)
'''

'\n##to combine word for "tidak"\nfor i in range(len(df)):\n  if \'tidak\' in df[\'comment\']:\n    split_text = df[\'comment\'][i].split()\n    for a,b in enumerate(split_text):\n      if b == \'tidak\':\n        split_text[a:(a+2)] = [\'\'.join(split_text[a:(a+2)])]\n    df[\'comment\'][i] = \' \'.join(split_text)\n'

In [ ]:
listData_tidak = df['comment'].values.tolist()
for i in range(len(listData_tidak )):
  split_text = listData_tidak [i].split()
  #print(split_textare)
  for idx,sub in enumerate(split_text):
    if sub == 'tidak' or sub == 'Tidak':
      #print('yes')
      split_text[idx:(idx+2)] = [''.join(split_text[idx:(idx+2)])]
      #print(split_text)
  df['comment'][i] = ' '.join(split_text)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
listData_kurang = df['comment'].values.tolist()
for i in range(len(listData_kurang )):
  split_text = listData_kurang [i].split()
  #print(split_textare)
  for idx,sub in enumerate(split_text):
    if sub == 'kurang' or sub == 'Kurang':
      #print('yes')
      split_text[idx:(idx+2)] = [''.join(split_text[idx:(idx+2)])]
      #print(split_text)
  df['comment'][i] = ' '.join(split_text)

In [ ]:
df.head(30)

,channel,comment,time,like,pertanyaan
0,AutonetMagz,sah! kia sonet & nissan magnite sekarang punya...,7 months ago,1.3K,False
1,BocilKudet,"mirip mazda 2 all new yaa, mantap👍🏻",14 hours ago,0,False
2,Imas Sartika,semoga 4 tahun lagi 🤭🤭🤣 sholawatin aja dulu 🤲,17 hours ago,0,False
3,Ismail Abd Hakim,kembar ativa di malaysia,3 days ago,0,False
4,Bakul Iwak,"saya udah beli ini mobil, dan puas sama fitur,...",3 days ago,0,False
5,Bass martin,recommended buat wanita jdi kalau parkir bisa ...,7 days ago (edited),0,False
6,Ratu yusniar,bensin boros ga yaa ini,9 days ago,0,False
7,Marsel Ginting,minyaknya res om 😁,10 days ago,0,False
8,kopi holic's,amin amin smga kebeli,13 days ago,0,False
10,cas wito,leg room kiri supir dikasi tempat koin. kaki j...,2 weeks ago,0,False


In [ ]:
##Function to clean comment##
def cleanText(commenT):
   # nltk.download('stopwords')
    my_file = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/Cleaning data/combined_stop_words.txt", "r")
    content = my_file.read()
    stop_words = content.split("\n")
    file_2  = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/Cleaning data/update_combined_slang_words.txt", "r")
    content2 = file_2.read()
    slang_words = ast.literal_eval(content2)
    my_file.close()
    file_2.close()

    commenT = commenT.lower()
    commenT = re.sub(r':', '', commenT)
    commenT = re.sub(r'‚Ä¶', '', commenT)
    #replace consecutive non-ASCII characters with a space
    commenT = re.sub(r'[^\x00-\x7F]+',' ', commenT)

    #remove emojis from commenT
    #commenT = emoji_pattern.sub(r'', commenT)
    
    #remove punctuation manually
    commenT = re.sub('[^a-zA-Z]', ' ', commenT)
    
    #remove tags
    commenT=re.sub("&lt;/?.*?&gt;","&lt;&gt;",commenT)
    
    #remove digits and special chars
    commenT=re.sub("(\\d|\\W)+"," ",commenT)

    #remove other symbol from commenT
    commenT = re.sub(r'â', '', commenT)
    commenT = re.sub(r'€', '', commenT)
    commenT = re.sub(r'¦', '', commenT)

    word_tokens = word_tokenize(commenT)
    for w in word_tokens:
        if w in slang_words.keys():
            word_tokens[word_tokens.index(w)] = slang_words[w]

    #filter using NLTK library append it to a string
    filtered_commenT = [w for w in word_tokens if not w in stop_words]
    filtered_commenT = []

    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in string.punctuation:
            filtered_commenT.append(w.lower())
    return ' '.join(filtered_commenT)

df['clean_text'] = df['comment']
re_clean = df['clean_text'].apply(lambda x: cleanText(x))
df['clean_text'] = re_clean
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5208 entries, 0 to 5585
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel     5208 non-null   object
 1   comment     5208 non-null   object
 2   time        5208 non-null   object
 3   like        5208 non-null   object
 4   pertanyaan  5208 non-null   object
 5   clean_text  5208 non-null   object
dtypes: object(6)
memory usage: 444.8+ KB


In [ ]:
##Function to tokenize clean-text and##
def count_words(x):
    words = word_tokenize(x)
    #m= len(words)
    n=str(words)
    return n

df['word_length'] = df['clean_text'].apply(lambda x:count_words(x))
#df['word_length'].value_counts().sort_index()
df.head()

,channel,comment,time,like,pertanyaan,clean_text,word_length
0,AutonetMagz,sah! kia sonet & nissan magnite sekarang punya...,7 months ago,1.3K,False,sah kia sonet nissan magnite saingan duo astra,"['sah', 'kia', 'sonet', 'nissan', 'magnite', '..."
1,BocilKudet,"mirip mazda 2 all new yaa, mantap👍🏻",14 hours ago,0,False,mirip mazda all new yaa mantap,"['mirip', 'mazda', 'all', 'new', 'yaa', 'mantap']"
2,Imas Sartika,semoga 4 tahun lagi 🤭🤭🤣 sholawatin aja dulu 🤲,17 hours ago,0,False,semoga sholawatin,"['semoga', 'sholawatin']"
3,Ismail Abd Hakim,kembar ativa di malaysia,3 days ago,0,False,kembar ativa malaysia,"['kembar', 'ativa', 'malaysia']"
4,Bakul Iwak,"saya udah beli ini mobil, dan puas sama fitur,...",3 days ago,0,False,beli mobil puas fitur suspensi bentuk mobilnya...,"['beli', 'mobil', 'puas', 'fitur', 'suspensi',..."


In [ ]:
datasetSentimentPositif = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/positif.txt", "r")
contentPositif = datasetSentimentPositif.read()
positif = contentPositif.split("\n")
positif  = [each_string.lower() for each_string in positif]

datasetSentimentNegatif = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/negatif.txt", "r")
contentNegatif = datasetSentimentNegatif.read()
negatif = contentNegatif.split("\n")
negatif   = [each_string.lower() for each_string in negatif]

datasetSentimentswitch = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/switch.txt", "r")
contentswitch = datasetSentimentswitch.read()
switch = contentswitch.split("\n")
switch   = [each_string.lower() for each_string in switch]

print(positif)
print(negatif)
#print(switch)
df['Sentiment']=df['clean_text']

['bagus', 'cakep', 'segar', 'fresh', 'keren', 'mewah', 'mantab', 'mantap', 'mantul', 'gagah', 'cool', 'modern', 'futuristik', 'banyak fitur', 'milenial', 'murah', 'terjangkau', 'kebeli', 'beli', 'bisa beli', 'akan beli', 'lega', 'luas', 'kuat', 'nanjak', 'tangguh', 'besar', 'kuat', 'respon', 'garang', 'irit', 'unik', 'spesial', 'cool', 'suka', 'ada', 'best', 'trending', 'widih', 'good', 'luxury', 'moga', 'ke beli', 'isa tuku', 'semoga', 'rezeki', 'cash', 'pengen', 'kerenn', 'tjakepp', 'ngiler', 'smoga', 'naksir', 'doain', 'kebili', 'bisa beli', 'sempurna', 'kagum', 'ganteng', 'bisa beli', 'mantapp', 'desain garang', 'bisa memiliki', 'mantao', 'kereeen', 'bs kebeli', 'muantabb', 'impian', 'suka', 'tranding', 'ajib', 'cakepp', 'laku keras', 'good looking', 'mantabs', 'lengkap', 'jos', 'mtap', 'mau', 'kepengen', 'ngeri', 'coll', 'top', 'tooopp', 'kren', 'udah pesan', 'cantik', 'oke bgt', 'pengen tumbas', 'kereen', 'gila ', 'minat juga', 'ngeri banget', 'nice', 'buset', 'buseh', 'bungkus',

In [ ]:
#define positif =1 and negatif 0
list_data = df['Sentiment'].values.tolist()
for i in range(len(list_data)):
  split_textare = list_data[i].split()
  #print(split_textare)
  for idx,sub in enumerate(split_textare):
    if sub in positif:
      split_textare[idx]="1"
    if sub in negatif:
      split_textare[idx]="0"
  print(split_textare)
  df['Sentiment'][i] = ' '.join(split_textare)

Streaming output truncated to the last 5000 lines.
['diketawain', 'sonet', 'min', 'sonet', 'menang', 'fiturnya', 'tertawa']
['1', 'masuk', 'mobil', '1', 'banget', 'ampe', '0', 'banget', 'jalanan', 'tidak', 'batasnya', 'gerak', 'kendaraan', 'amiinnnn']
['mhardja', 'tv', 'pret']
['xl', 'is', '1']
['uda', 'abgret', 'bawaan', 'pabrik', 'saudara', 'laki-laki', 'terima', 'kasih']
['mah', '1', 'turbo', 'tss', 'teknologynya', 'bikin', '0', 'meragukan', 'ake', 'turbo', 'cc', '0', 'silinder', 'mesin', 'tahan', 'tuh']
['pengguna', 'youtube', 'dijegal', 'mindset', 'indo', 'mobil', '0', '0', 'mesin', 'ayla', 'harga', 'jt', 'seat', 'mendang', 'lebih', 'baik']
['dr', 'mrk', 'daihatsu', 'namanya']
['tri', 'widarto', 'aa']
['mhardja', 'tv', 'bukan', 'paham', 'otomotif', '1', 'mobil', 'mikirin', 'harga', 'jual', 'mah']
['mobil', 'jpg', 'pelan', 'turun', 'harga', 'ga', 'saingan', 'harganya', 'gila', 'an', 'fitur']
['pajak', 'nya', 'kira', 'kak']
['mhardja', 'tv', 'toyota', 'mesin', 'cc', 'silinder', 'har

In [ ]:
'''
list_data_hitung = df['Sentiment'].values.tolist()
hitung = 0
for i in range(len(list_data_hitung)):
  split_textar = list_data_hitung[i].split()
  #print(split_textar)
  int(hitung)
  for idx,sub in enumerate(split_textar):
    if sub == '1':
      #print(split_textar)
      #hitung[idx] = hitung[idx] + 1 
      #split_textar[idx] = split_textar[idx] + split_textar[idx] 
      hitung[idx] = sum(int(split_textar[idx]))     
      print(split_textar[idx]) 
    #if sub in '0':
      #hitung = hitung - 1
      #split_textar[idx]= hitung
  #print(split_textar)
  #df['word_length'][i] = ' '.join(split_textar)
'''

"\nlist_data_hitung = df['Sentiment'].values.tolist()\nhitung = 0\nfor i in range(len(list_data_hitung)):\n  split_textar = list_data_hitung[i].split()\n  #print(split_textar)\n  int(hitung)\n  for idx,sub in enumerate(split_textar):\n    if sub == '1':\n      #print(split_textar)\n      #hitung[idx] = hitung[idx] + 1 \n      #split_textar[idx] = split_textar[idx] + split_textar[idx] \n      hitung[idx] = sum(int(split_textar[idx]))     \n      print(split_textar[idx]) \n    #if sub in '0':\n      #hitung = hitung - 1\n      #split_textar[idx]= hitung\n  #print(split_textar)\n  #df['word_length'][i] = ' '.join(split_textar)\n"

#INPUT USER TO ADD AND DELETE WORD

In [ ]:
newfileName = input("Input Name File.csv :  ")
newfileName = str(newfileName)
df.to_csv(newfileName)

Input Name File.csv :  newFile.csv


In [ ]:
print('THIS IS LIST OF WORD THATS IN SLANG WORDS')
file_2  = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/Cleaning data/update_combined_slang_words.txt", "r")
content2 = file_2.read()
slang_words = ast.literal_eval(content2)
print(type(slang_words))
slang = pd.DataFrame(list(slang_words.items()))
slang


THIS IS LIST OF WORD THATS IN SLANG WORDS
<class 'dict'>


,0,1
0,@,di
1,abis,habis
2,ad,ada
3,adlh,adalah
4,afaik,as far as i know
...,...,...
1028,ngamuk,mengamuk
1029,iso,bisa
1030,mbuh,tidak tahu
1031,kzl,kesal


In [ ]:
class_list = dict() 
data = input('Enter name & score separated by ":" ') 
temp = data.split(':') 
class_list[temp[0]] = int(temp[1]) 
 
# Displaying the dictionary 
for key, value in class_list.items(): 
	print('Name: {}, Score: {}'.format(key, value)) 

Enter name & score separated by ":" aldo:aldi


ValueError: ignored

In [ ]:
input_string_Slang   = input('Enter elements of a list separated by space ')
listMesinBaru  = input_string_Slang.split() 
#print(user_list_interior)
Mesin.extend(listMesinBaru)

In [ ]:
print('THIS LIST OF WORD IN COMMENT')
results = Counter()
df['clean_text'].str.lower().str.split().apply(results.update)
listKata = pd.DataFrame(list(results.items()))
listKata = listKata.rename(columns={0:'KATA',1:'JUMLAH KATA'})
listKata = listKata.sort_values(by="JUMLAH KATA",ascending=False)
listKata.head(20)

THIS LIST OF WORD IN COMMENT


,KATA,JUMLAH KATA
60,mobil,568
44,om,555
1,toyota,446
2,raize,364
31,tidak,313
41,beli,300
314,fitra,284
71,cc,244
332,fitur,222
0,harga,212


In [ ]:
###Function to let user input delete word from key_list ###
def del_word(x,key_list):
    n = len(key_list)
    word_tokens = word_tokenize(x)
    new_x =''
    for word in word_tokens:
        if word not in key_list:
            new_x = new_x+word+' '
    return new_x

##Input For User##
input_string = input('Enter elements of a list separated by space ')
print("\n")
user_list = str(input_string.split()) 
df['clean_text'] = df['clean_text'].apply(lambda x: del_word(x,user_list))

Enter elements of a list separated by space nya ga




In [ ]:
datasetEksterior = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/eksterior.txt", "r")
contentEksterior = datasetEksterior.read()
Eksterior = contentEksterior.split("\n")
Eksterior  = [each_string.lower() for each_string in Eksterior]
#Eksterior  = pd.DataFrame(list(Eksterior.items()))


datasetInterior = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/interior.txt", "r")
contentInterior = datasetInterior.read()
Interior = contentInterior.split("\n")
Interior   = [each_string.lower() for each_string in Interior]
#Interior = pd.DataFrame(list(Interior.items()))

datasetMesin = open("/content/drive/My Drive/MBKM pak kris toyota/Dataset/mesin.txt", "r")
contentMesin = datasetMesin.read()
Mesin = contentMesin.split("\n")
Mesin   = [each_string.lower() for each_string in Mesin]
#Mesin  = pd.DataFrame(list(Mesin.items()))

print("THIS IS DATASET FOR INTERIOR")
print((Interior))
print("="*50)
print("THIS IS DATASET FOR EKSTERIROR")
print(Eksterior)
print("="*50)
print("THIS IS DATASET FOR MESIN")
print(Mesin)


THIS IS DATASET FOR INTERIOR
['dashboard', 'kursi/seat', 'audio', 'screen', 'tft', 'speedometer', 'kaca film', 'fitur', 'sound system', 'twitter', 'speaker', 'warna', 'steer', 'tombol', 'switch', 'ac', 'klakson', 'handel', 'saku/kantong', 'tempat botol', 'box/kotak', 'sandaran', 'trim', 'kulit', 'plastik', 'ruang', 'kabin', 'sein', 'sliding', '']
THIS IS DATASET FOR EKSTERIROR
['bodi', 'desain', 'style', 'warna', 'bumper', 'spoiler', 'grill', 'roof rail', 'rack', 'lampu', 'spion', 'aerokit', 'kit', 'velg', 'ban', 'tampang depan', 'model', 'dimensi', 'chrom/krom', 'tampak belakang', 'emblem', 'bentuk', '']
THIS IS DATASET FOR MESIN
['tenaga', 'power', 'torsi', 'cc', 'akselerasi', 'fuel/bb', 'transmisi', 'akselerasi', 'mesin', 'safety', 'kenyamanan', 'cvt', 'at', 'kecepatan', 'turbo', '']


In [ ]:
input_string_Eksterior = input('Enter elements of a list separated by space ')
listEksteriorBaru  = input_string_Eksterior.split() 
#print(user_list_interior)
Eksterior.extend(listEksteriorBaru)
print(Eksterior)

Enter elements of a list separated by space aku kamu anda
['bodi', 'desain', 'style', 'warna', 'bumper', 'spoiler', 'grill', 'roof rail', 'rack', 'lampu', 'spion', 'aerokit', 'kit', 'velg', 'ban', 'tampang depan', 'model', 'dimensi', 'chrom/krom', 'tampak belakang', 'emblem', 'bentuk', '', 'aku', 'kamu', 'dia', 'dan', 'anda', 'aku', 'kamu', 'anda']


In [ ]:
input_string_Interior   = input('Enter elements of a list separated by space ')
listInteriorBaru  = input_string_Interior.split() 
#print(user_list_interior)
Interior.extend(listInteriorBaru)
print(Interior)

Enter elements of a list separated by space kita mereka
['dashboard', 'kursi/seat', 'audio', 'screen', 'tft', 'speedometer', 'kaca film', 'fitur', 'sound system', 'twitter', 'speaker', 'warna', 'steer', 'tombol', 'switch', 'ac', 'klakson', 'handel', 'saku/kantong', 'tempat botol', 'box/kotak', 'sandaran', 'trim', 'kulit', 'plastik', 'ruang', 'kabin', 'sein', 'sliding', '', 'kita', 'mereka']


In [ ]:
input_string_Mesin   = input('Enter elements of a list separated by space ')
listMesinBaru  = input_string_Mesin.split() 
#print(user_list_interior)
Mesin.extend(listMesinBaru)
print(Mesin)

Enter elements of a list separated by space cc
['tenaga', 'power', 'torsi', 'cc', 'akselerasi', 'fuel/bb', 'transmisi', 'akselerasi', 'mesin', 'safety', 'kenyamanan', 'cvt', 'at', 'kecepatan', 'turbo', '', 'cc']
